In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "6"
os.environ["PATH"] += ":/home/natsuki/miniconda3/envs/ada/bin"
device = "cuda"
import pickle
import torch
import numpy as np
from PIL import Image, ImageDraw, ImageFont
from pathlib import Path
import json
#from multiprocessing.shared_memory import SharedMemory
from IPython.display import display, clear_output
from script_util import wrap_G

In [ ]:
from script_util import wrap_G
#cfg = "00030-v4-mirror-auto4-gamma100-batch64-noaug-resumecustom/network-snapshot-011289.pkl"
cfg = "00023-white_yc05_yw04-mirror-auto4-gamma10-noaug/network-snapshot-021800.pkl"
fm = Path(f"/data/natsuki/training116/{cfg}")
G = wrap_G(fm, device)

In [ ]:
G.synth(G.map())

In [ ]:
with open("/home/natsuki/illustration2vec/tag_list.json") as f:
    tag = json.load(f)
taginv = dict()
for i, t in enumerate(tag):
    taginv[t] = i

with open("/home/natsuki/bizarre-pose-estimator/_data/danbooru/_filters/intently_combatively_rules.json") as f:
    rules = json.load(f) 
ruleinv = dict()
for i, rule, in enumerate(rules):
    t = rule["name"]
    ruleinv[t] = i

In [ ]:
#, mmap_mode="r"
ws = np.load("/data/natsuki/fact_lib/ws.npy")

In [ ]:
# 4ms
%time biz = np.load("/data/natsuki/fact_lib/biz.npy")

In [ ]:
%time preds = np.load("/data/natsuki/fact_lib/preds.npy")
#%time preds = np.load(arr_path)

In [ ]:
# PCA
from sklearn.decomposition import FastICA, PCA, IncrementalPCA, MiniBatchSparsePCA, SparsePCA, KernelPCA
import itertools
# Standard PCA
class PCAEstimator():
    def __init__(self, n_components):
        self.n_components = n_components
        self.solver = 'full'
        self.transformer = PCA(n_components, svd_solver=self.solver)
        self.batch_support = False

    def get_param_str(self):
        return f"pca-{self.solver}_c{self.n_components}"

    def fit(self, X):
        self.transformer.fit(X)

        # Save variance for later
        self.total_var = X.var(axis=0).sum()

        # Compute projected standard deviations
        self.stdev = np.dot(self.transformer.components_, X.T).std(axis=1)

        # Sort components based on explained variance
        idx = np.argsort(self.stdev)[::-1]
        self.stdev = self.stdev[idx]
        self.transformer.components_[:] = self.transformer.components_[idx]

        # Check orthogonality
        dotps = [np.dot(*self.transformer.components_[[i, j]])
            for (i, j) in itertools.combinations(range(self.n_components), 2)]
        if not np.allclose(dotps, 0, atol=1e-4):
            print('IPCA components not orghogonal, max dot', np.abs(dotps).max())

        self.transformer.mean_ = X.mean(axis=0, keepdims=True)

    def get_components(self):
        var_ratio = self.stdev**2 / self.total_var
        return self.transformer.components_, self.stdev, var_ratio
pca = PCAEstimator(64)
pca.fit(ws[:10**5])

cp, std, var = pca.get_components()
x = ws[:100].mean(axis=0)
# 0rotate 1洋服ロング 2横回転 45ショートヘア 6暗髪、ツインテ 7つり目 8服ビキニ 9ロング 10首をかしげる 11 above bellow 13 ロングケモミミ

F = lambda X: np.tanh(5*(X-.5))*.5+.5
ps = [-i/10 for i in range(10)] + [-i/10 for i in range(10)][::-1] + [i/10 for i in range(10)] + [-i/10 for i in range(10)]
for p in ps:
    clear_output(wait=True)
    i = 10
    y = x + 2* p * cp[i] * std[i]
    img = G.synth(torch.from_numpy(y[None,None,:][:,[0]*16,:]).to(device))
    display(img)

In [ ]:
from functools import lru_cache
@lru_cache(maxsize=None)
def search(index, thr=0.9):
    return np.where(preds[:, index] > thr)[0]

In [ ]:
"""予測率0.9以上のwベクトルの1000個の平均を記録する"""
font = ImageFont.truetype("/usr/share/fonts/opentype/ipafont-gothic/ipag.ttf", 50)
seedss = dict()
imgs = dict()
wms = dict()
missing = list()
for index, query in enumerate(tag):
    clear_output(wait=True)
    wm = 0
    searched = search(index, 0.9)
    seedss[query] = searched
    cnt = len(searched)
    if cnt == 0:
        missing.append(query)
        continue
    wm = torch.from_numpy(ws[searched[:1000]].mean(axis=0)[None,None,:][:,[0]*16,:]).to(device)
    img = G.synth(wm)
    ImageDraw.Draw(img).text((0,0), f"{query}\n{index}\n{cnt}", (0,0,0), font)
    wms[query] = wm
    imgs[query] = img
    display(img)
print(len(missing))

In [ ]:
for i, t in enumerate(tag):
    if " ears" in t:
        print(t, len(search(taginv[t], 0.9)))

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
t = "wolf ears" #334
t = "fox ears" #376
t = "cat ears" #89
t = "animal ears" #32
t = "1boy" #24
t = "glasses" #57
t = "hair over one eye" #281
t = "patchouli knowledge" #520
i = taginv[t]
#print(i, t)
#searched = search(i, 0.9)

from nokogiri.defaultdotdict import defaultdotdict
score = defaultdotdict(float)
for i, t in enumerate(tqdm(tag)):
    N = 1000
    searched = preds[:,i].argsort()[-N:]
    score.mean = preds[:,i][searched].mean()
    score.std = preds[:,i][searched].std()
    score.max = preds[:,i][searched].max()
    score.min = preds[:,i][searched].min()
    txt = " ".join(f"{k}={v:0.4f}" for k, v in score.items())
    with open(f"/data/natsuki/fact_lib/i2v_txt/{i}.txt", "w") as f:
        f.write(txt)
    wm_np = ws[searched].mean(axis=0)
    wm = torch.from_numpy(wm_np[None,None,:][:,[0]*16,:]).to(device)
    np.save(f"/data/natsuki/fact_lib/i2v_wm/{i}.npy", wm_np)
    img = G.synth(wm)
    img.save(f"/data/natsuki/fact_lib/i2v_img/{i}.jpg")

In [ ]:
imgs["1boy"]

In [ ]:
imgs["patchouli knowledge"]

In [ ]:
F = lambda X: np.tanh(5*(X-.5))*.5+.5
ps = [1/i for i in range(1, 10+1)]
ps = ps[::-1] + [1]*10 +ps
ps = [p for p in ps]
arrs = list()
for p in ps:
    clear_output(wait=True)
    #img, arr = G.synth(wms["izayoi sakuya"] + p*(wms["animal ears"]-wms["safe"]), retarr=True)
    #img, arr = G.synth(wms["patchouli knowledge"] + p*(wms["glasses"]-wms["safe"]), retarr=True)
    #img, arr = G.synth(wms["hatsune miku"] + p*(wms["animal ears"]-wms["safe"]), retarr=True)
    img, arr = G.synth(wms["izayoi sakuya"] + p*(wms["closed eyes"]-wms["safe"]), retarr=True)
    arrs.append(arr)
    display(img)

In [ ]:
import imageio
from subprocess import check_output
dname = "megane.gif"
imageio.mimsave(dname, arrs, fps=10)
print(check_output(f"du -sh {dname}", shell=True).decode(), end="")
print(check_output(f"mogrify -layers 'optimize' -fuzz 3% -loop 0 {dname}", shell=True).decode(), end="")
print(check_output(f"du -sh {dname}", shell=True).decode(), end="")

In [ ]:
for img in imgs.values:
    clear_output(wait=True)
    display(img)

In [ ]:
import ipyplot
ipyplot.plot_images(list(imgs.values())[700:], max_images=100)